In [1]:
import pickle
import pandas as pd
from explainers.dce import DistributionalCounterfactualExplainer
import os

In [2]:
data_path = 'data/hotel_booking'

In [3]:
factual = pd.read_csv(os.path.join(data_path,'factual.csv'))
counterfactual = pd.read_csv(os.path.join(data_path,'counterfactual.csv'))

In [4]:
factual

,lead_time,booking_changes,is_canceled
0,35,0,0.380332
1,87,0,0.446267
2,2,0,0.147630
3,48,2,0.093440
4,6,0,0.173178
5,12,0,0.217856
6,1,0,0.141751
7,162,0,0.493088
8,250,0,0.612698
9,165,0,0.496249


In [5]:
counterfactual

,lead_time,booking_changes,is_canceled
0,35.835574,-0.010555,0.384997
1,100.701171,-0.147769,0.481608
2,1.543849,-0.005207,0.145972
3,64.269392,1.937958,0.112608
4,-4.264569,0.009205,0.118994
5,3.608891,0.010795,0.155204
6,11.889563,-0.009178,0.219499
7,141.807265,0.312219,0.401004
8,241.418233,0.045382,0.583865
9,163.090685,0.004685,0.492971


In [6]:
factual['is_canceled'].mean()

0.32024891499999997

In [7]:
counterfactual['is_canceled'].mean()

0.309672289

In [14]:
factual.lead_time.mean()

76.8

In [15]:
counterfactual.lead_time.mean()

75.99000528079708